# MDL vs OOD Generalization - Results Analysis

This notebook analyzes the results of our experiment testing whether MDL regularization (β-VAE compression) improves out-of-distribution generalization in grid world navigation.

## Hypothesis
**Agents with stronger compression pressure (higher β in β-VAE) will show better OOD generalization performance**

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import wandb

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load results
results_path = Path("results.json")
if results_path.exists():
    with open(results_path, 'r') as f:
        results = json.load(f)
    print("✅ Results loaded successfully")
    print(f"Found results for {len(results)} agents")
else:
    print("❌ No results file found. Run train.py first.")
    results = {}

## 1. Training Performance Analysis

In [ ]:
if results:
    # Extract training curves
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    for agent_name, agent_results in results.items():
        training_metrics = agent_results['training_metrics']
        episodes = [m['episode'] for m in training_metrics]
        rewards = [m['total_reward'] for m in training_metrics]
        success_rates = [m['success'] for m in training_metrics]
        
        # Smooth curves with rolling average
        window = 100
        if len(rewards) >= window:
            rewards_smooth = pd.Series(rewards).rolling(window).mean()
            success_smooth = pd.Series(success_rates).rolling(window).mean()
        else:
            rewards_smooth = rewards
            success_smooth = success_rates
        
        beta = agent_results['agent_config']['beta']
        label = f"{agent_name} (β={beta})"
        
        # Plot training curves
        axes[0, 0].plot(episodes, rewards_smooth, label=label, alpha=0.8)
        axes[0, 1].plot(episodes, success_smooth, label=label, alpha=0.8)
        
        # Plot loss curves if available
        if 'reconstruction_loss' in training_metrics[0]:
            recon_losses = [m['reconstruction_loss'] for m in training_metrics]
            kl_losses = [m['kl_loss'] for m in training_metrics]
            
            axes[1, 0].plot(episodes, recon_losses, label=label, alpha=0.8)
            axes[1, 1].plot(episodes, kl_losses, label=label, alpha=0.8)
    
    axes[0, 0].set_title('Training Rewards')
    axes[0, 0].set_xlabel('Episode')
    axes[0, 0].set_ylabel('Total Reward')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].set_title('Success Rate')
    axes[0, 1].set_xlabel('Episode')
    axes[0, 1].set_ylabel('Success Rate')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].set_title('Reconstruction Loss')
    axes[1, 0].set_xlabel('Episode')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].set_title('KL Divergence Loss')
    axes[1, 1].set_xlabel('Episode')
    axes[1, 1].set_ylabel('KL Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 2. OOD Performance Analysis

In [ ]:
if results:
    # Create comparison dataframe
    comparison_data = []
    
    for agent_name, agent_results in results.items():
        beta = agent_results['agent_config']['beta']
        standard_success = agent_results['standard_success_rate']
        
        # Add standard performance
        comparison_data.append({
            'Agent': agent_name,
            'Beta': beta,
            'Scenario': 'Standard',
            'Success_Rate': standard_success,
            'Performance_Ratio': 1.0
        })
        
        # Add OOD performances
        ood_results = agent_results['ood_results']
        ood_ratios = agent_results['ood_performance_ratios']
        
        for key, success_rate in ood_results.items():
            if key.endswith('_success_rate'):
                scenario = key.replace('_success_rate', '')
                ratio_key = f"{scenario}_performance_ratio"
                ratio = ood_ratios.get(ratio_key, 0)
                
                comparison_data.append({
                    'Agent': agent_name,
                    'Beta': beta,
                    'Scenario': scenario.replace('_', ' ').title(),
                    'Success_Rate': success_rate,
                    'Performance_Ratio': ratio
                })
    
    df = pd.DataFrame(comparison_data)
    
    # Plot OOD performance comparison
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Success rates by scenario
    sns.barplot(data=df, x='Scenario', y='Success_Rate', hue='Agent', ax=axes[0])
    axes[0].set_title('Success Rates by Scenario')
    axes[0].set_ylabel('Success Rate')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Performance ratios (OOD vs Standard)
    ood_df = df[df['Scenario'] != 'Standard']
    sns.barplot(data=ood_df, x='Scenario', y='Performance_Ratio', hue='Agent', ax=axes[1])
    axes[1].set_title('OOD Performance Ratios (OOD/Standard)')
    axes[1].set_ylabel('Performance Ratio')
    axes[1].axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Baseline')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Display summary table
    print("\n📊 Performance Summary Table:")
    summary_table = df.pivot(index='Agent', columns='Scenario', values='Success_Rate')
    summary_table['Beta'] = df.groupby('Agent')['Beta'].first()
    summary_table = summary_table.sort_values('Beta')
    print(summary_table.round(3))

## 3. Hypothesis Testing

In [ ]:
if results:
    # Extract beta values and average OOD performance ratios
    agent_data = []
    
    for agent_name, agent_results in results.items():
        beta = agent_results['agent_config']['beta']
        
        # Calculate average OOD performance ratio
        ood_ratios = [v for k, v in agent_results['ood_performance_ratios'].items() 
                     if k.endswith('_performance_ratio')]
        avg_ood_ratio = np.mean(ood_ratios) if ood_ratios else 0
        
        agent_data.append({
            'agent': agent_name,
            'beta': beta,
            'avg_ood_ratio': avg_ood_ratio,
            'standard_success': agent_results['standard_success_rate']
        })
    
    agent_df = pd.DataFrame(agent_data)
    
    # Plot correlation
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Beta vs Average OOD Ratio
    axes[0].scatter(agent_df['beta'], agent_df['avg_ood_ratio'], s=100, alpha=0.7)
    for _, row in agent_df.iterrows():
        axes[0].annotate(row['agent'], (row['beta'], row['avg_ood_ratio']), 
                        xytext=(5, 5), textcoords='offset points')
    
    # Fit trend line
    z = np.polyfit(agent_df['beta'], agent_df['avg_ood_ratio'], 1)
    p = np.poly1d(z)
    axes[0].plot(agent_df['beta'], p(agent_df['beta']), "r--", alpha=0.8)
    
    axes[0].set_xlabel('Beta (Compression Pressure)')
    axes[0].set_ylabel('Average OOD Performance Ratio')
    axes[0].set_title('Beta vs OOD Generalization')
    axes[0].grid(True, alpha=0.3)
    
    # Beta vs Standard Performance (control)
    axes[1].scatter(agent_df['beta'], agent_df['standard_success'], s=100, alpha=0.7, color='orange')
    for _, row in agent_df.iterrows():
        axes[1].annotate(row['agent'], (row['beta'], row['standard_success']), 
                        xytext=(5, 5), textcoords='offset points')
    
    axes[1].set_xlabel('Beta (Compression Pressure)')
    axes[1].set_ylabel('Standard Success Rate')
    axes[1].set_title('Beta vs Standard Performance (Control)')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate correlations
    if len(agent_df) >= 2:
        corr_ood = np.corrcoef(agent_df['beta'], agent_df['avg_ood_ratio'])[0, 1]
        corr_standard = np.corrcoef(agent_df['beta'], agent_df['standard_success'])[0, 1]
        
        print(f"\n📈 Correlation Analysis:")
        print(f"Beta vs OOD Performance: {corr_ood:.3f}")
        print(f"Beta vs Standard Performance: {corr_standard:.3f}")
        
        print(f"\n🧪 Hypothesis Test Results:")
        if corr_ood > 0.5:
            print("✅ STRONG SUPPORT: Higher β significantly improves OOD generalization")
        elif corr_ood > 0.3:
            print("✅ MODERATE SUPPORT: Higher β appears to improve OOD generalization")
        elif corr_ood < -0.3:
            print("❌ HYPOTHESIS REJECTED: Higher β appears to hurt OOD generalization")
        else:
            print("❓ INCONCLUSIVE: No clear relationship found")
        
        # Check if effect is specific to OOD (not just general performance)
        if abs(corr_ood) > abs(corr_standard) + 0.2:
            print("🎯 SPECIFIC EFFECT: The relationship is stronger for OOD than standard performance")
        else:
            print("⚠️  GENERAL EFFECT: The relationship affects both OOD and standard performance similarly")

## 4. Compression Analysis

In [ ]:
if results:
    # Analyze compression ratios and their relationship to performance
    compression_data = []
    
    for agent_name, agent_results in results.items():
        config = agent_results['agent_config']
        obs_dim = config['obs_dim']
        latent_dim = config['latent_dim']
        beta = config['beta']
        
        compression_ratio = obs_dim / latent_dim
        avg_ood_ratio = np.mean([v for k, v in agent_results['ood_performance_ratios'].items() 
                                if k.endswith('_performance_ratio')])
        
        compression_data.append({
            'agent': agent_name,
            'beta': beta,
            'obs_dim': obs_dim,
            'latent_dim': latent_dim,
            'compression_ratio': compression_ratio,
            'avg_ood_ratio': avg_ood_ratio
        })
    
    comp_df = pd.DataFrame(compression_data)
    
    print("🗜️ Compression Analysis:")
    print(comp_df[['agent', 'beta', 'compression_ratio', 'avg_ood_ratio']].round(3))
    
    # Plot compression vs performance
    plt.figure(figsize=(10, 6))
    plt.scatter(comp_df['compression_ratio'], comp_df['avg_ood_ratio'], 
               s=comp_df['beta']*50, alpha=0.7, c=comp_df['beta'], cmap='viridis')
    
    for _, row in comp_df.iterrows():
        plt.annotate(f"{row['agent']}\n(β={row['beta']})", 
                    (row['compression_ratio'], row['avg_ood_ratio']), 
                    xytext=(5, 5), textcoords='offset points')
    
    plt.xlabel('Compression Ratio (Obs Dim / Latent Dim)')
    plt.ylabel('Average OOD Performance Ratio')
    plt.title('Compression vs OOD Performance\n(Bubble size = β value)')
    plt.colorbar(label='Beta Value')
    plt.grid(True, alpha=0.3)
    plt.show()

## 5. Conclusions and Next Steps

In [ ]:
if results:
    print("📋 EXPERIMENT CONCLUSIONS:")
    print("=" * 50)
    
    # Summary statistics
    best_ood_agent = max(agent_data, key=lambda x: x['avg_ood_ratio'])
    best_standard_agent = max(agent_data, key=lambda x: x['standard_success'])
    
    print(f"🏆 Best OOD Performance: {best_ood_agent['agent']} (β={best_ood_agent['beta']}, ratio={best_ood_agent['avg_ood_ratio']:.3f})")
    print(f"🏆 Best Standard Performance: {best_standard_agent['agent']} (β={best_standard_agent['beta']}, success={best_standard_agent['standard_success']:.3f})")
    
    print("\n🔬 KEY FINDINGS:")
    print(f"• Correlation between β and OOD performance: {corr_ood:.3f}")
    print(f"• Correlation between β and standard performance: {corr_standard:.3f}")
    
    print("\n🚀 NEXT STEPS:")
    print("• Test with more diverse OOD scenarios")
    print("• Experiment with different latent dimensions")
    print("• Compare with other compression methods (PCA, ICA)")
    print("• Analyze learned representations qualitatively")
    print("• Test on more complex environments")
    
    print("\n📝 THEORY CARD UPDATE:")
    if corr_ood > 0.3:
        print("✅ Update MDL_PRINCIPLE.md with positive results")
        print("✅ Consider this principle for further investigation")
    else:
        print("❌ Update MDL_PRINCIPLE.md with negative/inconclusive results")
        print("🔄 Consider revising hypothesis or experimental setup")
else:
    print("No results to analyze. Please run the training script first.")